# 基于细粒度特征的热红外目标追踪 Part 2

## 定义热红外图像的细粒度特征

图像细粒度的相关特征在区分干扰因素方面有较为优秀的表现，并且卷积神经网络的浅卷积层，如第三层卷积层，包含了较为详细的图像信息。可以利用卷积神经网络的这一特性，在浅卷积层去定义、计算热红外图像的细粒度特征。

*Reference:*

Liu Q, Li X, He Z, et al. Multi-Task Driven Feature Models for Thermal InfraredTracking[C]//2020 Thirty-Fourth AAAI Conference on Artificial Intelligence. AAAI, 2020.

Vaswani A, Shazeer N, Parmar N, et al. Attention is all you need[C]//Advances in neural information processing systems. 2017: 5998-6008.

Wang X, Girshick R, Gupta A, et al. Non-local neural networks[C]//Proceedings of the IEEE conference on computer vision and pattern recognition. 2018: 7794-7803.

本文提出的基于细粒度特征的热红外目标追踪框架如下所示：

![image1.png](./image1.png)

热红外图像的细粒度信息对于区分热红外跟踪目标至关重要，因为类内的热红外目标通常具有相似的语义模式，比如两个穿不同衣服的人，在热红外图像中具有相似的视觉模式。因此，细粒度的特征可以被利用于挖掘目标局部细节上的特征。Efﬁcient Convolution Operators (ECO) 是近几年 VOT 挑战赛中表现较好的追踪器之一，其算法中利用预训练好的深度神经网络VGG-net来提取图像的特征，并且仅选用了2个卷积层的输出与原始图像合并进行训练与测试。在 StructSiam 追踪算法中，追踪器也是通过2层浅卷积层来检测、提取目标的局部特征。在参考文献中，热红外图像在经过卷积神经网络的浅卷积层，以第3层卷积层为例，能够包含较为详细的图像信息；在第5层卷积层后能够得到热红外图像的高区分度特征。如图所示，右上方为输入的热红外图像，输出的分别为经过第3层卷积层后的图像以及经过第五层卷积层后的图像。相对于第5层，第3层卷积层输出有更大的空间以及更多的信息能够被利用来生成鉴别性更强特征，使得跟踪器能够更好的对跟踪目标进行识别。

![image2.png](./image2.png)

在卷积神经网络提取特征的过程当中，基于图像的局部相关性，输入至网络中图像的每一个位置都只与卷积核大小的邻域内的其他特征进行交互。尽管卷积神经网络相较于全连接网络极大地减少了参数数量和运算量，但同时也失去对全局信息的整合能力。正是基于该点，本文考虑在浅层卷积层后添加新的结构来对全局信息进行整合。

注意力（Attention）机制目前被广泛用于自然语言处理当中，在计算机视觉领域，注意力机制能够用于构建非局部神经网络（Non-local neural networks），挖掘连续帧像素级之间的关系。在参考文献中，Wang等人注意到Vaswani等人提出的自注意力模型（self-attention module）是非局部操作中的一种特殊形式。Wang等人定义的非局部操作为：
$$y_i=\frac{1}{C(x)}\sum_{\forall j}{f(x_i, x_j) g(x_j)}$$
公式的含义为：对于输出的特征y中的某个位置 i，通过函数 f 计算输入的 x 中 i 处的特征与其他可能的特征点位置 j 处的特征的两两关系，同时使用一个一元函数 g 计算 x 在 j 处的特征表示，最后将两个函数的所有输出对应位置加权求和，再除以系数 C(x) 进行归一化。

在参考文献中，Wang等人指出当函数g为简单的线性映射$g(x_j)=\mathbf{W}_gx_j$，函数f为高斯函数$f(x_i, x_j)=e^{{\theta(x_i)}^T\phi(x_j)}$，其中$\theta(x_i)=\mathbf{W}_\theta x_i$、$\phi(x_j)=\mathbf{W}_\phi x_j$，系数C(x)为$\sum_{\forall j}f(x_i, x_j)$时，对于给定的位置 i,定义式中$\frac{1}{C(x)}f(x_i, x_j)$即是沿着维度 j 的 softmax 计算，因而有：$y=\mathrm{softmax}(x^T \mathbf{W}_\theta^T \mathbf{W}_\phi x) g(x)$。这正是Vaswani等人提出的自注意力模型中Scaled dot-product attention的形式。

通过以上研究，本文将自注意力形式的非局部操作模块衔接在浅层神经网络之后，定义热红外图像 $\mathbf{X}\in \Bbb{R}^{H \times W \times C}$ 的细粒度特征为：$$\omega(\mathbf{X}) = f(\phi_{conv3}(\mathbf{X}))$$ 其中，$\phi_{conv3}(\cdot)$ 通过三层卷积层提取出图像的局部特征，$f(\cdot)$ 为非局部操作模块。整体结构如图所示：

![image3.png](./image3.png)

图中，$\otimes$表示batch上的矩阵乘法，$\oplus$表示单位加操作。

令 $\mathbf{Z}\in \Bbb{R}^{H \times W \times C}$ 是图像经过三层卷积层后得到的特征图，我们首先计算出特征图上像素之间的关联图$S\in \Bbb{R}^{HW \times HW}$: $$s_{ij}=\frac{exp\ (\mathbf{W}_Q z_i \otimes \mathbf{W}_K z_j)}{\sum_{n=1}^{H \times W}exp\ (\mathbf{W}_Q z_i \otimes \mathbf{W}_K z_n)}$$

其中，$\mathbf{W}_Q$、$\mathbf{W}_K$是两个$1\times1$的卷积层；$z$是$\mathbf{Z}$上的一个特征点，$\mathbf{Z}=\{z_i\}_{i=1}^{H \times W}$。将关联图映射在特征图上可以得到图像的像素级关联特征，也是非局部操作的输出y：$$y=\sum_{i=1}^{H \times W}\sum_{j=1}^{H \times W} s_{ij} (\mathbf{W}_V z_j)$$

其中，$\mathbf{W}_V$是一个$1\times1$的卷积层。最后我们将图像的像素级关联特征与浅卷积层得到的原特征图通过类似于残差连接的方式得到我们想要的细粒度特征：$$f(\mathbf{Z})=\mathbf{Z}+\delta y$$

其中，$\delta$作为权重矩阵能够通过训练被自动赋值。得到的细粒度特征将拥有3通道，被送入SiamRPN++框架进行处理。